# Downloading simulation data from AWS 

October 18, 2021

In this notebook, Vince and Rashad are trying to use Matt's example code to download some simulation data of our choosing



In [7]:
import mdtraj as md
import itertools
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

import os, urllib, subprocess, glob
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def run_cmd(cmd):
    subprocess.check_output(cmd, stderr=subprocess.STDOUT,shell=True).decode().split('\n')

In [10]:
### pull data down from AWS and post-process trajectories

# download xtc and (gro/top)

url_prefix = 'https://fah-public-data-covid19-absolute-free-energy.s3.us-east-2.amazonaws.com'
project = 14823
runs = range(1) # just RUN0
clones = range(1) # just CLONE0
for run in runs:
    for clone in clones:
        
        PRC_dir = f'data/P{project}_R{run}_C{clone}'
        if not os.path.exists(PRC_dir):
            os.makedirs(PRC_dir)
        download(f'{url_prefix}/setup_files/p{project}/RUN0/npt.gro',
          f'data/P{project}_R{run}_C{clone}/npt.gro')
        download(f'{url_prefix}/setup_files/p{project}/RUN0/topol.top',
          f'data/P{project}_R{run}_C{clone}/topol.top')
        gen = 0
        while True:
            try:
                print(f'\nProcessing P{project}_R{run}_C{clone}_G{gen}')
                download(f'{url_prefix}/PROJ{project}/RUN{run}/CLONE{clone}/results{gen}/traj_comp.xtc',
                  f'data/P{project}_R{run}_C{clone}/traj_comp.xtc')
            except Exception as e:
                print(e)
                break
            
            path = f'data/P{project}_R{run}_C{clone}'
            
            ### WARNING:  This next section needs an installation of gmx in your path to work!!!!
            
            # Step 1: Build a custom *.tpr for the subset of atoms (molecules "LIG" and "system1") in the *.xtc trajectories
            
            ## write a dummy *.mdp for minimization (we will need a tpr for trjconv)
            write_mdp_cmd = f'echo "integrator          = steep" > {path}/xtc.mdp'
            run_cmd(write_mdp_cmd)
            
            ## make an index file for just the atoms in the xtc
            """Example:
              0 System              : 54272 atoms
              1 Other               :    64 atoms
              2 LIG                 :    64 atoms
              3 NA                  :    30 atoms
              4 CL                  :    30 atoms
         ---> 5 Protein             :  4657 atoms
              6 Protein-H           :  2352 atoms
            """
            make_index_cmd = f'echo "5|2\nq\n" | gmx make_ndx -f {path}/npt.gro -o {path}/index.ndx'
            run_cmd(make_index_cmd)
            
            # make a (.top) for xtc atoms, omitting the last three lines:
            """
            [ molecules ]
            ; Compound       #mols
            LIG                  1
            system1              1
     omit X HOH              16497
     omit X NA                  30
     omit X CL                  30
            """
            fin = open(f'{path}/topol.top', 'r')
            topol_lines = fin.readlines()
            fin.close()
            fout = open(f'{path}/xtc.top', 'w')
            fout.writelines(topol_lines[:-3])
            fout.close()
            
            # write a *.gro file for just the xtc atoms
            make_xtcgro_cmd = f'echo "24\n" | gmx editconf -f {path}/npt.gro -n {path}/index.ndx -o {path}/xtc.gro'
            run_cmd(make_xtcgro_cmd)

            # write a *.ndx file for just the xtc atoms
            make_xtcndx_cmd = f'echo "3|2\nq\n" | gmx make_ndx -f {path}/xtc.gro -o {path}/xtc.ndx'
            run_cmd(make_xtcndx_cmd)
            
            # gmx grompp to make a fake *.tpr
            make_xtctpr_cmd = f'gmx grompp -f {path}/xtc.mdp -c {path}/xtc.gro -p {path}/xtc.top -o {path}/xtc.tpr'
            run_cmd(make_xtctpr_cmd)
            
            # gmx trjconv for PBC correction
            pbc_correct_cmd = f'echo "3\n14\n" | gmx trjconv -f {path}/traj_comp.xtc -s {path}/xtc.tpr -n {path}/xtc.ndx -pbc mol -center -o {path}/traj_{str(gen).zfill(4)}.xtc'
            run_cmd(pbc_correct_cmd)
            
            # for cmd in [write_mdp_cmd, make_index_cmd, make_xtctop_cmd, make_xtcgro_cmd,
            #    make_xtcndx_cmd, make_xtctpr_cmd, pbc_correct_cmd]:
            #    subprocess.check_output(cmd, stderr=subprocess.STDOUT,shell=True).decode().split('\n')

            #for cmd in [write_mdp_cmd, make_index_cmd]:
            #    subprocess.check_output(cmd, stderr=subprocess.STDOUT,shell=True).decode().split('\n')

                
                
            
            traj = md.load(f'{path}/traj_{str(gen).zfill(4)}.xtc',top = f'{path}/xtc.gro')
            PHE140_indices = [a.index for a in traj.topology.atoms if a.residue.index in [141] and a.name in ['CG','CD1','CD2','CE1','CE2','CZ']]
            HIS163_indices = [a.index for a in traj.topology.atoms if a.residue.index in [164]and a.name in ['CG','ND1','CD2','CE1','NE2']]
            traj_PHE140_indices = traj.atom_slice(PHE140_indices)
            traj_HIS163_indices = traj.atom_slice(HIS163_indices)
            coords_PHE140_com = md.compute_center_of_mass(traj_PHE140_indices)
            coords_HIS163_com = md.compute_center_of_mass(traj_HIS163_indices)
            hacked_traj = traj

            ## creating hacked traj 0 and 1
            hacked_traj.xyz[:,0,:] = coords_PHE140_com # PHE140 trajectory
            hacked_traj.xyz[:,1,:] = coords_HIS163_com # HIS163 trajectory


            ## computing the distance between the center of mass of the PHE140 and HIS163 ring
            PHE140_HIS163_distances = md.compute_distances(hacked_traj, [[0,1]])[:,0]
            np.save(f'{path}/PHE140_HIS163_distnces_G{str(gen).zfill(4)}', PHE140_HIS163_distances)

            gen += 1
        
            # file_list = ['traj_comp.xtc','xtc.mdp','xtc.top','xtc.ndx','xtc.gro','xtc.tpr','index.ndx']
            file_list = ['traj_comp.xtc','xtc.mdp','xtc.top','xtc.ndx','xtc.tpr']
            for file in glob.glob(f'{path}/*'):
                if any(substring in file for substring in file_list):
                    os.remove(file)
                    
            # remove backups:
            for file in glob.glob(f'{path}/#*'):
                os.remove(file)
                    
           #gen += 1


npt.gro: 2.45MB [00:00, 7.37MB/s]                            
topol.top: 2.65MB [00:00, 9.76MB/s]                            
traj_comp.xtc: 213kB [00:00, 1.18MB/s]                            



Processing P14823_R0_C0_G0


traj_comp.xtc:   4%|▍         | 8.19k/207k [00:00<00:04, 44.2kB/s]


Processing P14823_R0_C0_G1


traj_comp.xtc: 213kB [00:00, 969kB/s]                             
traj_comp.xtc: 213kB [00:00, 1.11MB/s]                            

...


Processing P14823_R0_C0_G231



traj_comp.xtc:   4%|▍         | 8.19k/203k [00:00<00:04, 48.5kB/s]


Processing P14823_R0_C0_G232


traj_comp.xtc: 205kB [00:00, 1.00MB/s]                            
traj_comp.xtc: 0.00B [00:00, ?B/s]


Processing P14823_R0_C0_G233
HTTP Error 404: Not Found


In [11]:
topol_lines

[';\n',
 ';   File MS03262020_RL/MS03262020_LIG1/topol.top  was generated\n',
 ';   By user: tug27224 (40003)\n',
 ';   On host: g008\n',
 ';   At date: Sat. March  6 03:11:15 2020\n',
 ';\n',
 ';   This is a standalone topology file\n',
 ';\n',
 ';   Created by:\n',
 ';   ParmEd:       simulate.py, VERSION 3.2.0\n',
 ';   Executable:   simulate.py\n',
 ';   Library dir:  /usr/local/gromacs/share/gromacs/top\n',
 ';   Command line:\n',
 ';     simulate.py 1 100 0\n',
 ';\n',
 '\n',
 '[ defaults ]\n',
 '; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ\n',
 '1               2               no              1            0.83333333  \n',
 '\n',
 '[ atomtypes ]\n',
 '; name    at.num    mass    charge ptype  sigma      epsilon\n',
 'C1             6  12.010780  0.00000000  A     0.33996695       0.359824\n',
 'C2             6  12.010780  0.00000000  A     0.33996695      0.4577296\n',
 'N1             7  14.006720  0.00000000  A     0.32499985        0.71128\n',
 'O1         

In [12]:
line_indices_to_grab = []
for j in range(len(topol_lines)):
    if topol_lines[j].count('ParmEd') > 0:
        line_indices_to_grab.append(j)

print('line_indices_to_grab', line_indices_to_grab)

line_indices_to_grab [9]


In [16]:
my_grofile = f'data/P{project}_R{run}_C{clone}/npt.gro'
fin = open(my_grofile, 'r')
lines = fin.readlines()